In [ ]:
#i "nuget:C:\Users\dicolomb\OneDrive - Microsoft\LocalNugetDev"

In [ ]:
#r "nuget:piTop"
#r "nuget:piTop.FoundationKit"

In [ ]:
using PiTopMakerArchitecture.Foundation;
using PiTopMakerArchitecture.Foundation.Components;
using PiTopMakerArchitecture.Foundation.Sensors;
using System.Reactive.Linq;
using System.Reactive.Disposables;

In [ ]:
var _disposables = new CompositeDisposable();

In [ ]:
var plate = new Plate();